# Concurrent Agents

In this section, we explore the use of multiple agents working concurrently. We cover three main patterns:

1. **Single Message & Multiple Processors**  
   Demonstrates how a single message can be processed by multiple agents subscribed to the same topic simultaneously.

2. **Multiple Messages & Multiple Processors**  
   Illustrates how specific message types can be routed to dedicated agents based on topics.

3. **Direct Messaging**  
   Focuses on sending messages between agents and from the runtime to agents.

In [39]:
import asyncio
from dataclasses import dataclass

from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.base import AgentId, MessageContext, TopicId
from autogen_core.components import (
    ClosureAgent,
    ClosureContext,
    DefaultSubscription,
    DefaultTopicId,
    RoutedAgent,
    default_subscription,
    message_handler,
    type_subscription,
)

In [40]:
@dataclass
class Task:
    task_id: str


@dataclass
class TaskResponse:
    task_id: str
    result: str

## Single Message & Multiple Processors
The first pattern shows how a single message can be processed by multiple agents simultaneously:

- Each `Processor` agent subscribes to the default topic using the {py:meth}`~autogen_core.components.default_subscription` decorator.
- When publishing a message to the default topic, all registered agents will process the message independently.
```{note}
Below, we are subscribing `Processor` using the {py:meth}`~autogen_core.components.default_subscription` decorator, there's an alternative way to subscribe an agent without using decorators altogether as shown in [Subscribe and Publish to Topics](../framework/message-and-communication.ipynb#subscribe-and-publish-to-topics), this way the same agent class can be subscribed to different topics.
```


In [129]:
@default_subscription
class Processor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"{self._description} starting task {message.task_id}")
        await asyncio.sleep(2)  # Simulate work
        print(f"{self._description} finished task {message.task_id}")

In [130]:
runtime = SingleThreadedAgentRuntime()

await Processor.register(runtime, "agent_1", lambda: Processor("Agent 1"))
await Processor.register(runtime, "agent_2", lambda: Processor("Agent 2"))

runtime.start()

await runtime.publish_message(Task(task_id="task-1"), topic_id=DefaultTopicId())

await runtime.stop_when_idle()

Agent 1 starting task task-1
Agent 2 starting task task-1
Agent 1 finished task task-1
Agent 2 finished task task-1


## Multiple messages & Multiple Processors
Second, this pattern demonstrates routing different types of messages to specific processors:
- `UrgentProcessor` subscribes to the "urgent" topic
- `NormalProcessor` subscribes to the "normal" topic

We make an agent subscribe to a specific topic type using the {py:meth}`~autogen_core.components.type_subscription` decorator.

In [122]:
@type_subscription(topic_type="urgent")
class UrgentProcessor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"Urgent processor starting task {message.task_id}")
        await asyncio.sleep(1)  # Simulate work
        print(f"Urgent processor finished task {message.task_id}")


@type_subscription(topic_type="normal")
class NormalProcessor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"Normal processor starting task {message.task_id}")
        await asyncio.sleep(3)  # Simulate work
        print(f"Normal processor finished task {message.task_id}")

After registering the agents, we can publish messages to the "urgent" and "normal" topics:

In [117]:
runtime = SingleThreadedAgentRuntime()

await UrgentProcessor.register(runtime, "urgent_processor", lambda: UrgentProcessor("Urgent Processor"))
await NormalProcessor.register(runtime, "normal_processor", lambda: NormalProcessor("Normal Processor"))

runtime.start()

await runtime.publish_message(Task(task_id="normal-1"), topic_id=TopicId(type="normal", source="default"))
await runtime.publish_message(Task(task_id="urgent-1"), topic_id=TopicId(type="urgent", source="default"))

await runtime.stop_when_idle()

Normal processor starting task normal-1
Urgent processor starting task urgent-1
Urgent processor finished task urgent-1
Normal processor finished task normal-1


## Direct Messages

In contrast to the previous patterns, this pattern focuses on direct messages. Here we demonstrate two ways to send them:

- Direct messaging between agents  
- Sending messages from the runtime to specific agents  

Things to consider when using direct messaging:  

- Messages are addressed using the {py:class}`~autogen_core.components.AgentId`.  
- The sender can expect to receive a response from the target agent.  
- We register the `WorkerAgent` class only once; however, we send tasks to two different workers.  
    - As stated in [Agent lifecycle](../core-concepts/agent-identity-and-lifecycle.md#agent-lifecycle), when delivering a message using an {py:class}`~autogen_core.components.AgentId`, the runtime will either fetch the instance or create one if it doesn't exist. In this case, the runtime creates two instances of workers when sending those two messages.  

In [49]:
class WorkerAgent(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> TaskResponse:
        print(f"{self.id} starting task {message.task_id}")
        await asyncio.sleep(2)  # Simulate work
        print(f"{self.id} finished task {message.task_id}")
        return TaskResponse(task_id=message.task_id, result=f"Results by {self.id}")


@default_subscription
class DelegatorAgent(RoutedAgent):
    def __init__(self, description: str, worker_type: str):
        super().__init__(description)
        self.worker_instances = [AgentId(worker_type, f"{worker_type}-1"), AgentId(worker_type, f"{worker_type}-2")]

    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> TaskResponse:
        print(f"Delegator received task {message.task_id}.")

        subtask1 = Task(task_id="task-part-1")
        subtask2 = Task(task_id="task-part-2")

        worker1_result, worker2_result = await asyncio.gather(
            self.send_message(subtask1, self.worker_instances[0]), self.send_message(subtask2, self.worker_instances[1])
        )

        combined_result = f"Part 1: {worker1_result.result}, " f"Part 2: {worker2_result.result}"
        task_response = TaskResponse(task_id=message.task_id, result=combined_result)

        await self.publish_message(task_response, DefaultTopicId())
        return task_response

In [50]:
runtime = SingleThreadedAgentRuntime()

await WorkerAgent.register(runtime, "worker", lambda: WorkerAgent("Worker Agent"))
await DelegatorAgent.register(runtime, "delegator", lambda: DelegatorAgent("Delegator Agent", "worker"))

runtime.start()

delegator = AgentId("delegator", "default")
response = await runtime.send_message(Task(task_id="main-task"), delegator)

print(f"Final result: {response.result}")
await runtime.stop_when_idle()

Delegator received task main-task.
worker/worker-1 starting task task-part-1
worker/worker-2 starting task task-part-2
worker/worker-1 finished task task-part-1
worker/worker-2 finished task task-part-2
Final result: Part 1: Results by worker/worker-1, Part 2: Results by worker/worker-2


Above, we demonstrated that when sending a message from the runtime to a specific agent (delegator), we can expect a response.

Alternatively, instead of sending a message directly to the `DelegatorAgent`, we can publish a message to the default topic. Since the `DelegatorAgent` is subscribed to that topic, it will process the message. However, note that when publishing messages, any responses from the receiving agent are discarded. In other words, the `publish_message` method will always return `None`.

To address this, we can use a {py:class}`~autogen_core.components.ClosureAgent`. Below is an example of how to collect responses using a {py:class}`~autogen_core.components.ClosureAgent`:  

In [48]:
queue = asyncio.Queue[TaskResponse]()


async def collect_result(_agent: ClosureContext, message: Task | TaskResponse, ctx: MessageContext) -> None:
    if isinstance(message, TaskResponse):
        await queue.put(message)


runtime.start()

await ClosureAgent.register_closure(
    runtime, "collect_result", collect_result, subscriptions=lambda: [DefaultSubscription()]
)

await runtime.publish_message(Task(task_id="main-task"), topic_id=DefaultTopicId())

await runtime.stop_when_idle()

Delegator received task main-task.
worker/worker-1 starting task task-part-1
worker/worker-2 starting task task-part-2
worker/worker-1 finished task task-part-1
worker/worker-2 finished task task-part-2


In [38]:
while not queue.empty():
    print(await queue.get())

TaskResponse(task_id='main-task', result='Part 1: Results by worker/worker-1, Part 2: Results by worker/worker-2')


If you're interested in more about direct messaging, check out the [Mixture of Agents](../design-patterns/mixture-of-agents.ipynb) pattern, which relies heavily on this approach.
